In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from crewai import LLM

llm = LLM("gpt-4.1-nano",temperature=0.01)

llm.call(
    "What is the capital of France?")

'The capital of France is Paris.'

In [8]:
email_with_bad_abbreviation="""
Subject: Fw: URGENT: Pls Rply ASAP
Hi,
I hope this email finds you well. I am writing to follow up on the previous email I sent regarding the project deadline. It is very important that we receive your feedback as soon as possible so that we can proceed with the next steps.
As you know, the UJRNT project is critical for our success, and we need your input to ensure that we are on the right track. Please let us know your thoughts on the matter at your earliest convenience.

Best,
Thanos
"""

In [ ]:
from crewai.tools import BaseTool

class ReplaceJargonTool(BaseTool):
    name: str = "replace_jargon"
    description: str = "Replaces jargon or abbreviations in an email with more professional language."

    def _run(self, email: str) -> str:
        # Example implementation that replaces common jargon with more professional terms
        replacements = {
            "pls": "please",
            "rply": "reply",
            "asap": "as soon as possible",
            "thx": "thank you",
            "btw": "by the way",
            "ujrnt" : "unifed joint research network team",
        }
        suggestions = []
        email_lower = email.lower()
        for jargon in replacements.keys():
            if jargon in email_lower:
                suggestions.append(f"Replace '{jargon}' with '{replacements[jargon]}'")
        #return the list as a string
        if suggestions:
            return "Consider the following suggestions to improve your email:\n" + "\n".join(suggestions)
        else:
            return "No jargon found in the email."

jt = ReplaceJargonTool()
jt.run(email_with_bad_abbreviation)

Using Tool: replace_jargon


"Consider the following suggestions to improve your email:\nReplace 'pls' with 'please'\nReplace 'rply' with 'reply'\nReplace 'asap' with 'as soon as possible'"

In [ ]:
from crewai import Agent, Task, Crew

email_assistant = Agent(
    role="Email_Assistant",
    goal="Improve email messages to make them more effective.",
    backstory="You are an email assistant that helps users improve their email messages. You can suggest changes to the subject line, body, and other parts of the email to make it more effective.",
    verbose=True,
    llm=llm
)

email_task = Task(
    description="Improve the following email message to make it more effective:{\n\n" + email_with_bad_abbreviation + "\n}",
    agent=email_assistant,
    expected_output="A professional and effective email message that improves the original email.",
)

crew = Crew(
    agents=[email_assistant],
    tasks=[email_task],
    verbose=True,
)

results = crew.kickoff()
results